In [1]:
from pycaret.classification import *
import pandas as pd
import os
import wandb

os.chdir('/home/antoine/gene_pheno_pred')
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["WANDB_API_KEY"]="4e5748d6c6f3917c78cdc38a516a1bac776faf58"
wandb.login()
wandb.init(
    project="cigap-classif",
)


wandb: Currently logged in as: antoine-toffano (mlb). Use `wandb login --relogin` to force relogin


In [2]:
# Load training set
train = pd.read_csv('/home/antoine/gene_pheno_pred/ConvKB_2023-04-13_16-59-47_train.csv' , header=0)
train = train.replace('no_link_known', 1)
train = train.replace(to_replace=r'^((?!no_link_known).)*$', value=0, regex=True)
train

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,ground_truth
0,-0.106307,-0.076018,-0.081182,0.028428,0.006551,-0.049328,-0.131857,-0.037628,0.003007,0.085964,...,-0.088359,-0.011955,0.072061,-0.239798,0.282087,-0.085467,0.008205,0.139863,-0.100712,0
1,-0.012138,-0.009232,0.024595,-0.021252,-0.015880,-0.047778,-0.011917,0.028311,-0.034624,0.059547,...,-0.274189,-0.501134,0.305044,0.213702,-0.469372,0.195861,-0.185408,0.056212,0.258238,0
2,-0.052939,-0.071929,0.021859,0.011558,-0.049912,-0.007417,0.011347,0.026139,-0.013402,-0.028613,...,-0.009154,-0.019550,-0.094600,0.098562,-0.025860,-0.037455,-0.112670,0.071711,0.070507,1
3,-0.733260,0.043293,0.119687,-0.123384,-0.071939,-0.126068,0.032904,0.110734,-0.096527,-0.095115,...,-0.101405,0.220706,0.192686,-0.176124,-0.216842,-0.436630,0.103777,-0.110369,-0.125698,1
4,-0.036974,-0.005872,-0.002371,-0.035732,-0.029971,-0.052257,0.016698,0.027884,0.000931,0.016802,...,0.005399,0.063239,-0.027632,-0.001155,0.048586,0.052991,0.027887,0.004721,-0.061660,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364605,-0.052012,-0.050973,0.035910,-0.059969,-0.061255,-0.053517,1.033800,0.041240,-0.052969,0.050190,...,0.034043,0.038958,-0.019946,-0.008046,0.001575,-0.011575,-0.047649,-0.035770,-0.002342,1
2364606,-0.059736,-0.061631,0.070624,0.048631,0.038553,0.030114,-0.824042,0.028283,-0.084142,-0.113815,...,-0.266709,-0.537886,0.321153,0.181186,-0.507742,0.207627,-0.238616,0.106768,0.250761,0
2364607,0.036065,0.071069,-0.168564,0.064844,0.088171,0.013884,0.160834,-0.273891,0.120310,-0.068251,...,0.009662,0.028640,-0.053873,0.002138,0.008572,0.031187,-0.037569,-0.022562,0.013139,0
2364608,0.059826,0.027907,0.042077,-0.105650,-0.061793,0.070876,0.088055,-0.013670,-0.131062,0.035856,...,-0.252920,0.035881,0.221206,-0.067258,-0.011880,-0.220015,-0.025483,0.051174,-0.045477,0


In [3]:
# Validation set
val = pd.read_csv('/home/antoine/gene_pheno_pred/ConvKB_2023-04-13_16-59-47_val.csv' , header=0)
val = val.replace('no_link_known', 1)
val = val.replace(to_replace=r'^((?!no_link_known).)*$', value=0, regex=True)
val

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,ground_truth
0,0.176603,0.144554,-0.121246,0.352947,0.161330,0.119488,0.204768,-0.100077,0.156833,-0.120513,...,0.048977,0.036158,-0.062647,0.061191,0.034371,0.017874,-0.018670,-0.061726,-0.032668,1
1,0.230500,0.027735,-0.311731,-0.056029,0.097707,0.272195,0.340572,0.002959,0.306213,-0.027642,...,0.039863,0.051089,-0.081835,0.056872,0.049417,0.033689,-0.008890,-0.049069,-0.015917,0
2,0.195384,0.188668,-0.220960,-0.023338,-0.027151,0.020286,0.266949,-0.348298,0.268543,-0.163545,...,0.022186,-0.007695,-0.020189,-0.036016,0.016186,0.028108,-0.007122,0.038059,-0.028528,1
3,0.026929,0.074788,0.035858,-0.005145,-0.042590,0.019052,0.033178,0.090141,-0.043595,-0.071027,...,0.167388,0.088925,-0.027883,-0.040423,0.085831,0.083911,-0.083440,-0.009200,-0.090322,0
4,0.008397,-0.001974,-0.007151,-0.004110,-0.033697,-0.053781,-0.049135,0.044767,-0.010571,0.032441,...,0.009489,0.020247,-0.027801,0.050793,0.010482,0.012848,-0.005974,-0.019075,-0.045278,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234137,0.176400,0.218860,-0.280247,0.305255,0.101689,0.111522,-0.095150,-0.109428,0.155395,-0.027120,...,0.036530,0.021950,-0.020570,-0.017535,0.031310,0.024575,-0.009150,-0.023717,-0.029366,1
234138,0.064738,0.126745,-0.028705,0.136302,0.126094,0.199326,0.161574,-0.012139,0.113478,-0.176163,...,0.046814,0.006897,-0.067952,-0.032035,-0.598163,0.024000,-0.018999,-0.023364,-0.046112,0
234139,0.009338,0.000967,0.000705,-0.058703,-0.058091,-0.004134,0.016403,0.012446,-0.016351,-0.027040,...,-0.064294,0.002646,-0.086266,0.143449,0.015991,-0.137966,0.122271,-0.188965,0.189511,1
234140,0.044665,-0.104824,-0.071804,-0.174067,-0.122643,-0.149357,-0.144405,0.002902,-0.149952,0.087081,...,-0.013302,0.045505,0.034258,0.093059,0.120011,0.007771,-0.001258,-0.075868,-0.069958,1


In [4]:
# Embeddings validation test set
embval = pd.read_csv('/home/antoine/gene_pheno_pred/ConvKB_2023-04-13_16-59-47_test.csv' , header=0)
embval = embval.replace('no_link_known', 1)
embval = embval.replace(to_replace=r'^((?!no_link_known).)*$', value=0, regex=True)
embval

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,ground_truth
0,0.143444,0.093716,-0.137825,-0.124671,-0.096162,0.134430,0.258746,-0.316311,0.063991,-0.294353,...,0.006072,-0.018314,-0.036506,0.058785,0.032657,0.068184,-0.035997,-0.134148,0.025432,0
1,0.018331,0.045232,0.001957,0.000281,-0.007401,0.909883,0.018149,-0.002059,0.016732,-0.009270,...,0.023204,0.026533,-0.035074,-0.007182,0.016385,0.017070,-0.040005,-0.024660,-0.013184,0
2,0.004819,0.054722,0.001181,0.086243,0.076688,0.046515,0.041210,-0.023142,0.060256,-0.125768,...,0.027227,0.041565,-0.049319,-0.046994,0.046374,0.048004,-0.061171,0.001339,-0.120643,0
3,-0.049837,-0.037773,0.039730,-0.042049,-0.032324,-0.035344,0.037563,-0.021121,-0.032801,-0.031846,...,0.033551,0.026903,-0.041563,0.005378,0.066259,0.040746,-0.047484,-0.029613,-0.017699,0
4,0.088384,0.012557,-0.111223,-0.101479,-0.090372,-0.079046,-0.071125,-0.055374,-0.095753,0.117984,...,-0.012302,-0.038869,-0.055369,0.045610,-0.033408,0.062304,-0.035480,-0.049748,-0.004965,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233779,0.028791,0.064465,0.016604,-0.005840,-0.009796,0.008425,0.013725,-0.001496,-0.007599,-0.075763,...,-0.004248,0.102976,-0.020468,-0.024157,-0.978574,0.016163,0.027392,-0.110037,-0.073671,1
233780,-0.004770,-0.048431,0.002252,-0.001596,-0.046143,-0.110508,-0.025915,0.009208,0.006273,0.032961,...,-0.211187,-0.086243,0.064273,-0.023515,-0.043269,-0.097919,0.059382,0.175372,0.076994,1
233781,0.027764,0.056585,-0.033077,-0.022086,-0.067494,-0.015891,-0.032308,0.012968,0.015531,-0.080184,...,0.051418,0.101561,-0.006042,-0.104236,0.101335,-0.052880,0.089030,0.016790,-0.083716,1
233782,0.078459,0.079812,-0.076346,-0.083066,-0.081467,-0.066998,-0.050276,-0.002377,-0.093573,-0.045859,...,0.021252,0.069637,-0.038760,0.034591,0.018672,0.037403,-0.031772,-0.048327,-0.011563,0


In [5]:
# Experiment setup
s = setup(train, target = 'ground_truth', fold_strategy = 'stratifiedkfold', fold=10, train_size = 0.8, n_jobs=-1, system_log=True, use_gpu = True, log_experiment='wandb')
exp = ClassificationExperiment()

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recomp

,Description,Value
0,Session id,1847
1,Target,ground_truth
2,Target type,Binary
3,Original data shape,"(2364610, 101)"
4,Transformed data shape,"(2364610, 101)"
5,Transformed train set shape,"(1891688, 101)"
6,Transformed test set shape,"(472922, 101)"
7,Numeric features,100
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


# K Neighbors Classifier

In [ ]:
knn = create_model('knn')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:22:34
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Classifier


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
plot_model(knn, plot = 'confusion_matrix')

In [ ]:
plot_model(knn, plot = 'error')

In [ ]:
plot_model(knn, plot = 'parameter')

In [ ]:
knn_val_pred = predict_model(knn, data=val)

In [ ]:
knn_val_pred = predict_model(knn, data=embval)

In [ ]:
save_model(knn, 'models_classif/convkb/binary_emb_train/knn')

# Naive Bayes

In [ ]:
nb = create_model('nb')

In [ ]:
plot_model(nb, plot = 'confusion_matrix')

In [ ]:
plot_model(nb, plot = 'error')

In [ ]:
plot_model(nb, plot = 'parameter')

In [ ]:
nb_val_pred = predict_model(nb, data=val)

In [ ]:
nb_val_pred = predict_model(nb, data=embval)

In [ ]:
save_model(nb, 'models_classif/convkb/binary_emb_train/nb')

# Decision Tree Classifier

In [ ]:
dt = create_model('dt')

In [ ]:
plot_model(dt, plot = 'confusion_matrix')

In [ ]:
plot_model(dt, plot = 'error')

In [ ]:
plot_model(dt, plot = 'parameter')

In [ ]:
dt_val_pred = predict_model(dt, data=val)

In [ ]:
dt_val_pred = predict_model(dt, data=embval)

In [ ]:
save_model(dt, 'models_classif/convkb/binary_emb_train/dt')

# SVM - Linear Kernel

In [ ]:
svm = create_model('svm')

In [ ]:
plot_model(svm, plot = 'confusion_matrix')

In [ ]:
plot_model(svm, plot = 'error')

In [ ]:
plot_model(svm, plot = 'parameter')

In [ ]:
svm_val_pred = predict_model(svm, data=val)

In [ ]:
svm_val_pred = predict_model(svm, data=embval)

In [ ]:
save_model(svm, 'models_classif/convkb/binary_emb_train/svm')

# Ridge Classifier

In [ ]:
ridge = create_model('ridge')

In [ ]:
plot_model(ridge, plot = 'confusion_matrix')

In [ ]:
plot_model(ridge, plot = 'error')

In [ ]:
plot_model(ridge, plot = 'parameter')

In [ ]:
ridge_val_pred = predict_model(ridge, data=val)

In [ ]:
ridge_val_pred = predict_model(ridge, data=embval)

In [ ]:
save_model(ridge, 'models_classif/convkb/binary_emb_train/ridge')

# Random Forest

In [ ]:
rf = create_model('rf')

In [ ]:
plot_model(rf, plot = 'confusion_matrix')

In [ ]:
plot_model(rf, plot = 'error')

In [ ]:
plot_model(rf, plot = 'parameter')

In [ ]:
rf_val_pred = predict_model(rf, data=val)

In [ ]:
rf_val_pred = predict_model(rf, data=embval)

In [ ]:
save_model(rf, 'models_classif/convkb/binary_emb_train/rf')

# Quadratic Discriminant Analysis

In [ ]:
qda = create_model('qda')

In [ ]:
plot_model(qda, plot = 'confusion_matrix')

In [ ]:
plot_model(qda, plot = 'error')

In [ ]:
plot_model(qda, plot = 'parameter')

In [ ]:
qda_val_pred = predict_model(qda, data=val)

In [ ]:
qda_val_pred = predict_model(qda, data=embval)

In [ ]:
save_model(qda, 'models_classif/convkb/binary_emb_train/qda')

# Linear Discriminant Analysis

In [ ]:
lda = create_model('lda')

In [ ]:
plot_model(lda, plot = 'confusion_matrix')

In [ ]:
plot_model(lda, plot = 'error')

In [ ]:
plot_model(lda, plot = 'parameter')

In [ ]:
lda_val_pred = predict_model(lda, data=val)

In [ ]:
lda_val_pred = predict_model(lda, data=embval)

In [ ]:
save_model(lda, 'models_classif/convkb/binary_emb_train/lda')

# Extra Trees

In [ ]:
et = create_model('et')

In [ ]:
plot_model(et, plot = 'confusion_matrix')

In [ ]:
plot_model(et, plot = 'error')

In [ ]:
plot_model(et, plot = 'parameter')

In [ ]:
et_val_pred = predict_model(et, data=val)

In [ ]:
et_val_pred = predict_model(et, data=embval)

In [ ]:
save_model(et, 'models_classif/convkb/binary_emb_train/et')

# Light Gradient Boosting Machine

In [ ]:
lightgbm = create_model('lightgbm')

In [ ]:
plot_model(lightgbm, plot = 'confusion_matrix')

In [ ]:
plot_model(lightgbm, plot = 'error')

In [ ]:
plot_model(lightgbm, plot = 'parameter')

In [ ]:
lgbm_val_pred = predict_model(lightgbm, data=val)

In [ ]:
lgbm_val_pred = predict_model(lightgbm, data=embval)

In [ ]:
save_model(lightgbm, 'models_classif/convkb/binary_emb_train/lightgbm')

# MLP Classifier

In [ ]:
mlp = create_model('mlp')

In [ ]:
plot_model(mlp, plot = 'confusion_matrix')

In [ ]:
plot_model(mlp, plot = 'error')

In [ ]:
plot_model(mlp, plot = 'parameter')

In [ ]:
mlp_val_pred = predict_model(mlp, data=val)

In [ ]:
mlp_val_pred = predict_model(mlp, data=embval)

In [ ]:
save_model(mlp, 'models_classif/convkb/binary_emb_train/mlp')